In [ ]:
# !pip install python-dotenv

In [18]:
import pandas as pd

from dotenv import load_dotenv
import os

import mysql.connector
from mysql.connector import Error

In [21]:
load_dotenv()

True

In [ ]:
df = pd.read_csv("data\\naveen_hostpital.csv")


In [3]:
df.head()

,City Name,Zone Name,Ward Name,Zone No.,Ward No.,Facility Name,Type (Hospital / Nursing Home / Lab),Class : (Public / Private),Pharmacy Available : Yes/No,Number of Beds in Emergency Wards,Number of Beds in facility type,Number of Doctors / Physicians,Number of Nurses,Number of Midwives Professional,Average Monthly Patient Footfall,Ambulance Service Available,Count of Ambulance
0,Pune,kasba - vishrambagwada WO,Ambil Odha,5.0,NaN,"Late Matoshri Ramabai Ambedkar Maternity Home,...",Hospital (Maternity Home),Public,Yes,0,10,2,4,1,4500,Yes,1
1,Pune,Aundh - Baner WO,Aundh,2.0,NaN,"Aundh Kuti Maternity Home, Aundh",Hospital (Maternity Home),Public,Yes,0,14,2,5,2,2500,Yes,1
2,Pune,Aundh - Baner WO,Bopodi,2.0,NaN,Late Draupadabai Murlidhar Khedekar Maternity ...,Hospital (Maternity Home),Public,Yes,0,16,2,4,9,1900,Yes,1
3,Pune,Ghole Road - Shivaji Nagar WO,Shivaji Nagar Station,2.0,NaN,"Dr. Dalvi, PMC Joint Project",Hospital (Maternity Home),Public,Yes,0,30,4,7,9,2400,Yes,2
4,Pune,Hadapsar - Mundhwa WO,Hadapsar,4.0,NaN,"Late Anna Saheb Magar Maternity Home, Hadapsar",Hospital (Maternity Home),Public,Yes,0,20,2,3,6,4500,Yes,1


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 737 entries, 0 to 736
Data columns (total 17 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   City Name                              737 non-null    object 
 1   Zone Name                              705 non-null    object 
 2   Ward Name                              735 non-null    object 
 3   Zone No.                               692 non-null    float64
 4   Ward No.                               0 non-null      float64
 5   Facility Name                          737 non-null    object 
 6   Type  (Hospital / Nursing Home / Lab)  737 non-null    object 
 7    Class : (Public / Private)            737 non-null    object 
 8   Pharmacy Available : Yes/No            737 non-null    object 
 9   Number of Beds in Emergency Wards      737 non-null    object 
 10  Number of Beds in facility type        737 non-null    int64  
 11  Number

In [6]:
df.columns

Index(['City Name', 'Zone Name', 'Ward Name', 'Zone No.', 'Ward No.',
       'Facility Name', 'Type  (Hospital / Nursing Home / Lab)',
       ' Class : (Public / Private)', 'Pharmacy Available : Yes/No',
       'Number of Beds in Emergency Wards ', 'Number of Beds in facility type',
       'Number of Doctors / Physicians', 'Number of Nurses',
       'Number of Midwives Professional ', 'Average Monthly Patient Footfall',
       'Ambulance Service Available', 'Count of Ambulance'],
      dtype='object')

In [7]:
df = df.where(pd.notnull(df), None)

In [12]:
df = df.fillna(0)

In [23]:

try:
    # Connect to MySQL Database
    connection = mysql.connector.connect(
        host = os.getenv("HOST"),
        database = os.getenv("DB_NAME"),
        user = os.getenv("USER"),
        password = os.getenv("PASS"),
    )

    if connection.is_connected():
        db_info = connection.get_server_info()
        print("Connected to MySQL Server version", db_info)
        cursor = connection.cursor()
        cursor.execute("SELECT DATABASE();")
        record = cursor.fetchone()
        print("You're connected to database:", record)

except Error as e:
    print("Error while connecting to MySQL", e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version 8.0.40
You're connected to database: ('naveen',)
MySQL connection is closed


In [ ]:
try:
    # Connect to MySQL Database
    connection = mysql.connector.connect(
        host=os.getenv("HOST"),
        database=os.getenv("DB_NAME"),
        user=os.getenv("USER"),
        password=os.getenv("PASS"),
    )

    if connection.is_connected():
        db_info = connection.get_server_info()
        print("Connected to MySQL Server version", db_info)
        cursor = connection.cursor()
        cursor.execute("SELECT DATABASE();")
        record = cursor.fetchone()
        print("You're connected to database:", record)

        for _, row in df.iterrows():
            placeholders = ', '.join(['%s'] * len(row))
            columns = ', '.join(f"`{col.rstrip()}`" for col in df.columns)
            print(columns)
            sql = f"INSERT INTO healthcare_facilities VALUES ({placeholders})"
            cursor.execute(sql, tuple(row))
            print(sql, " Done ")

        connection.commit()
        print("Data inserted successfully into 'healthcare_facilities'.")


except Error as e:
    print("Error while connecting to MySQL", e)

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")

Connected to MySQL Server version 8.0.40
You're connected to database: ('naveen',)
`City Name`, `Zone Name`, `Ward Name`, `Zone No.`, `Ward No.`, `Facility Name`, `Type  (Hospital / Nursing Home / Lab)`, ` Class : (Public / Private)`, `Pharmacy Available : Yes/No`, `Number of Beds in Emergency Wards`, `Number of Beds in facility type`, `Number of Doctors / Physicians`, `Number of Nurses`, `Number of Midwives Professional`, `Average Monthly Patient Footfall`, `Ambulance Service Available`, `Count of Ambulance`
INSERT INTO healthcare_facilities VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)  Done 
`City Name`, `Zone Name`, `Ward Name`, `Zone No.`, `Ward No.`, `Facility Name`, `Type  (Hospital / Nursing Home / Lab)`, ` Class : (Public / Private)`, `Pharmacy Available : Yes/No`, `Number of Beds in Emergency Wards`, `Number of Beds in facility type`, `Number of Doctors / Physicians`, `Number of Nurses`, `Number of Midwives Professional`, `Average Monthly Patient